In [ ]:
import os, cv2
import string
import os.path as osp
import numpy as np
from collections import defaultdict
from vlmeval.smp import ls, load, dump, download_file, encode_image_file_to_base64, md5, mrlines
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing as mp
from PIL import Image, ImageFont, ImageDraw

font_URL = 'http://opencompass.openxlab.space/utils/Fonts/timesb.ttf'
font_file = 'timesb.ttf'
if not osp.exists(font_file):
    download_file(font_URL)
    
test_split_URL = 'https://s3-us-east-2.amazonaws.com/prior-datasets/ai2d_test_ids.csv'
test_split_file = 'ai2d_test_ids.csv'
if not osp.exists(test_split_file):
    download_file(test_split_URL)
    
test_ids = set(mrlines(test_split_file))
    
def proper_font_size(font_file, wh, text, ratio=1):
    font_size = 2
    while True:
        font = ImageFont.truetype(font_file, font_size)
        real_box = font.getbbox(text)
        real_wh = (real_box[2] - real_box[0], real_box[3] - real_box[1])
        if real_wh[0] > wh[0] * ratio or real_wh[1] > wh[1] * ratio:
            break
        font_size += 1
    return font_size

def cover_image(ann_path):
    data = load(ann_path)
    texts = list(data['text'].values())
    raw_img = ann_path.replace('annotations', 'images').replace('.json', '')
    tgt_img = raw_img.replace('images', 'images_abc')
    img = Image.open(raw_img)
    draw = ImageDraw.Draw(img)
    for text in texts:
        st, ed = tuple(text['rectangle'][0]), tuple(text['rectangle'][1])
        T = text['replacementText']
        draw.rectangle((st, ed), fill='white')
        font_size = proper_font_size(font_file, (ed[0] - st[0], ed[1] - st[1]), T, ratio=1)
        font = ImageFont.truetype(font_file, font_size)
        text_box = font.getbbox(T)
        text_wh = (text_box[2] - text_box[0], text_box[3] - text_box[1])
        cx, cy = (st[0] + ed[0]) // 2, st[1]
        stx = cx - text_wh[0] // 2
        sty = cy - text_wh[1] // 2
        draw.text((stx, sty), T, font=font, fill='black')
    img.save(tgt_img)    

In [ ]:
# Process for no mask images
test_ids = set(mrlines(test_split_file))

def detect_image_color(image):
    gray_image = image.convert('L')
    mean_brightness = np.mean(np.array(gray_image))
    if mean_brightness < 127:
        return 'white'
    else:
        return 'black'

def cover_image(ann_path):
    data = load(ann_path)
    texts = list(data['text'].values())
    raw_img = ann_path.replace('annotations', 'images').replace('.json', '')
    tgt_img = raw_img.replace('images', 'images_abc')
    img = Image.open(raw_img)
    draw = ImageDraw.Draw(img)
    color = detect_image_color(img)
    font_size = 0
    for text in texts:
        st, ed = tuple(text['rectangle'][0]), tuple(text['rectangle'][1])
        font_size += (ed[1] - st[1])
    if len(texts) != 0:
        font_size /= len(texts)
    else:
        font_size = 2
    for text in texts:
        st, ed = tuple(text['rectangle'][0]), tuple(text['rectangle'][1])
        T = text['replacementText']
        for i in range(2):
            draw.rectangle(
                [(st[0] - i, st[1] - i), (ed[0] + i, ed[1] + i)],
                outline=color
            )
        font = ImageFont.truetype(font_file, font_size)
        text_box = font.getbbox(T)
        text_wh = (text_box[2] - text_box[0], text_box[3] - text_box[1])
        cx, cy = (st[0] + ed[0]) // 2, st[1]
        stx = cx - text_wh[0] // 2
        sty = cy - text_wh[1] * 1.5
        if sty < 0:
            sty = cy + text_wh[1] * 1.3
        draw.text((stx, sty), T, font=font, fill=color)
    img.save(tgt_img)  

In [ ]:
download_file('https://ai2-public-datasets.s3.amazonaws.com/diagrams/ai2d-all.zip')
os.system('unzip -o ai2d-all.zip')

images = ls('ai2d/images/')
questions = ls('ai2d/questions/')
annotations = ls('ai2d/annotations/')
cates = load('ai2d/categories.json')

In [ ]:
pool = mp.Pool(32)
pool.map(cover_image, annotations)

In [ ]:
def puncproc(inText):
    import re
    outText = inText
    punct = [
        ';', r'/', '[', ']', '"', '{', '}', '(', ')', '=', '+', '\\', '_', '-',
        '>', '<', '@', '`', ',', '?', '!'
    ]
    commaStrip = re.compile('(\d)(,)(\d)')  # noqa: W605
    periodStrip = re.compile('(?!<=\d)(\.)(?!\d)')  # noqa: W605
    for p in punct:
        if (p + ' ' in inText or ' ' + p in inText) or (re.search(commaStrip, inText) is not None):
            outText = outText.replace(p, '')
        else:
            outText = outText.replace(p, ' ')
    outText = periodStrip.sub('', outText, re.UNICODE)
    return outText

def check_choices(line):
    def ischar(s):
        s = str(s)
        if s in ['{}', 'Both', 'None of above']:
            return True
        elif s.startswith('Stage ') and ischar(s[6:]):
            return True
        elif ' and ' in s and np.all([ischar(x) for x in s.split(' and ')]):
            return True
        elif len(s) <= 2:
            return True
        elif len(puncproc(s).split()) > 1:
            return np.all([ischar(x) for x in puncproc(s).split()])
        return False
    n_char = sum([ischar(line[x]) for x in 'ABCD'])
    return n_char >= 3

def check_question(question):
    words = puncproc(question).split()
    for ch in string.ascii_lowercase + string.ascii_uppercase:
        if ch in words:
            return True
    return False

def is_abc(abc, choices, question):
    if abc == 0:
        return False
    if check_choices(choices):
        return True
    if check_question(question):
        return True
    return False

In [ ]:
data_all = defaultdict(list)
for qfile in questions:
    data = load(qfile)
    idx = data['imageName'].split('.')[0]
    if idx not in test_ids:
        continue
    image_pth = qfile.replace('questions', 'images').replace('.json', '')
    cate = cates[image_pth.split('/')[-1]]
    for q, qmeta in data['questions'].items():
        assert '.png-' in qmeta['questionId']
        main, sub = qmeta['questionId'].split('.png-')
        idx = int(main) * 100 + int(sub)
        
        answers = qmeta['answerTexts']
        correct = qmeta['correctAnswer']
        
        data_all['index'].append(idx)
        data_all['question'].append(q)
        assert len(answers) == 4
        for c, a in zip('ABCD', answers):
            data_all[c].append(a)
        data_all['answer'].append('ABCD'[qmeta['correctAnswer']])
        data_all['category'].append(cate)
        data_all['abcLabel'].append(qmeta['abcLabel'])
        abc = is_abc(qmeta['abcLabel'], {x: data_all[x][-1] for x in 'ABCD'}, q)
        # if qmeta['abcLabel'] and not abc:
        #     print(qmeta['abcLabel'], {x: data_all[x][-1] for x in 'ABCD'}, q)
        data_all['image_path'].append(image_pth.replace('images', 'images_abc') if abc else image_pth)
data = pd.DataFrame(data_all)

In [ ]:
images = []
image_seen = {}
for idx, pth in zip(data['index'], data['image_path']):
    images.append(encode_image_file_to_base64(pth))

data['image'] = images
dump(data, 'AI2D_TEST.tsv')
print(md5('AI2D_TEST.tsv'))